In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp downloaderlabeler
# all_local

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
import pickle
import json
from steroidsornot.submissionshandler import SubmissionsHandler
from steroidsornot.database import Session
from steroidsornot.submission import Submission
from steroidsornot.firsttry import PrawClient

In [ ]:
from fastcore.test import *

In [ ]:
class DownloaderLabeler(object):
    def __init__(self, session):
        self.session = session
        self.unlabeled_submissions = session.query(Submission).filter_by(label=None)
        self.reddit = PrawClient().reddit()

        
        self._setup_buttons()
        
        self._main_loop()
        
    def _main_loop(self):
        for submission in self.unlabeled_submissions:
            data = json.loads(submission.data)
            reddit_submission = self.reddit.submission(url=data['full_link'])
            print(reddit_submission.title)
        
        self.session.close()

        
    def _setup_buttons(self):
        self.natural_button = Button(description='natural', layout=Layout(border = '2px solid green'))
        self.uncertain_button = Button(description='uncertain', layout=Layout(border = '2px solid brown'))
        self.steroids_button = Button(description='steroids', layout=Layout(border = '2px solid red'))
        self.irrelevant_button = Button(description='irrelevant', layout=Layout(border = '2px solid khaki'))
        self.box = widgets.HBox([self.natural_button,
                                 self.uncertain_button,
                                 self.steroids_button,
                                 self.irrelevant_button,
                                ])
        display(self.box)       

In [ ]:
#downloader_labeler = DownloaderLabeler(Session())

In [ ]:
session = Session()

submission = session.query(Submission).limit(1).first()

data = json.loads(submission.data)

2021-08-18 15:34:15,192 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-18 15:34:15,201 INFO sqlalchemy.engine.Engine SELECT submission.id AS submission_id, submission.label AS submission_label, submission.data AS submission_data 
FROM submission
 LIMIT ? OFFSET ?
2021-08-18 15:34:15,205 INFO sqlalchemy.engine.Engine [generated in 0.00353s] (1, 0)


In [ ]:
reddit = PrawClient().reddit()

Starting instance with client_id EKxTiNUwygf1WV5jlK0iew


In [ ]:
reddit_submission = reddit.submission(url=data['full_link'])
reddit_submission.title

'5\'8", 205lbs, Claims Natty'